In [53]:
# the following code allows you to import function from
# jupyter notebook in other directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [54]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb

IndexError: list index out of range

In [55]:
class ChiangOGWInvariant():
    def __init__(self, use_recursion_a=True, ogw_1_0_g2=var('x'), ogw_2_2=var('y'), debug=False):
        self.use_recursion_a = use_recursion_a
        self.ogw_1_0_g2 = ogw_1_0_g2
        self.ogw_1_0_g2 = -93/1024
        self.ogw_2_2 = ogw_2_2
        self.debug = debug

    #@functools.lru_cache(maxsize=None)
    def chiang_ogw_invariant(self, n, b, k, A, layer=0):
        #print(f"b:{b}, k:{k}, A:{A}")
        if isinstance(A, list):
            A = ClassList.compress(A, n)

        l = len(A)      
        if A.invalid_class_list:
            return 0    

        # wall-crossing theorem (Elad)
        if l > 0 and A[0] == -1:
            return -self.chiang_ogw_invariant(n, b, k+1, A[:-1]) 

        #degree axiom (Elad)
        if 2*sum(A) != (n - 3 + 2*b + k + 2*l - k*n):  
            # The maslov index is twice the chern class n+1 for holomorphic spheres. 
            # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
            # p 595 mcduff 
            #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
            if layer == 1:
                print("degree axiom fails")
            return 0    

        # other
        #if k == 0 and is_even(b):
        #    return 0    

        # zero axiom
        if b == 0:
            if k == 1 and l == 1 and A[0] == 0:
                return -1
            else:
                return 0    

        # unit axiom (May)
        # handle all beta = 0 cases
        if l > 0 and A[0] == 0:
            if b == 0 and k == 1 and l == 1:
                return -1
            else:
                return 0    

        # divisor axiom 
        if l > 0 and A[0] == 1:
            #A.pop(0)
            return (b/4)*self.chiang_ogw_invariant(n, b, k, A[1:])   

        # base cases
        if b == 1 and k == 1 and A == []:
            return 3    

        if b == 1 and k == 0 and A == [2]:
            return self.ogw_1_0_g2
        
        if b == 2 and k == 2 and A == []:
            return 2
        
        # TODO: hand check
        if b == 2 and k == 0 and A == [2, 2]:
            return (-1 + self.chiang_ogw_invariant(n, 2, 1, ClassList.compress([2], n))^2)\
                    /(self.chiang_ogw_invariant(n, 2, 2, ClassList.compress([], n)))
        
        # TODO : hand check
        if b == 2 and k == 1 and A == [2]:
            return -self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))\
                    *self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))
        
        if b == 2 and k == 0 and A == [3]:
            return 2*(self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([2, 2], n)) \
                    + self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))*\
                        self.chiang_ogw_invariant(n, 1, 0, ClassList.compress([1, 2], n)))
             

        # recursion (a)
        if l >= 2 and not (k >= 1 and not self.use_recursion_a):
            cor = self.chiang_ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
            col = 0
            ool = 0
            oor = 0
            A_0 = A[0]
            A_1 = A[1]  

            for t in A[2:].bipart():
                I1 = t[0]
                I2 = t[1]
                bipart_coeff = ClassList.bipart_coeff(A[2:], I1)    

                for i in range(0, n+1):
            
                    L1_cor = [1, A_1, i] + I1
                    L2_cor = [n-i, A_0-1] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        cor += bipart_coeff \
                            * (-1)^(b_*(n+1)/2) *  gw_invariant(n, b_, L1_cor) \
                            *  self.chiang_ogw_invariant(n, b1, k, L2_cor)   

                for i in range(0, n+1):
                
                    L1_col = [1, A[0] - 1, i] + I1 
                    L2_col = [n-i, A[1]] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        col += bipart_coeff \
                            * (-1)^((n+1)*b_/2)* gw_invariant(n, b_, L1_col) \
                            * self.chiang_ogw_invariant(n, b1, k, L2_col)    

                L1_ool = [1, A[0]-1] + I1
                L2_ool = [A[1]] + I2
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1
                        ool += bipart_coeff \
                            * binomial(k, k1)*self.chiang_ogw_invariant(n, b1, k1, L1_ool) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_ool)   

                L1_oor = [1, A[1]] + I1
                L2_oor = [A[0] - 1] + I2    
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1 

                        oor += bipart_coeff \
                            * binomial(k, k1) * self.chiang_ogw_invariant(n, b1, k1, L1_oor) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_oor)   

            result = cor - col - oor + ool
            if self.debug:
                print(f"recursion a: {result}")
            return result
        
        # recursion (c)
        if l >= 1 and k >= 1:
            print(f"used c, A:{A}")
            co = 0
            ool = 0
            oor = 0
            A_0 = A[0]  

            for t in A.bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A, I1)    

                for i in range(0, n+1):
                    for b1 in range(0, b): # make sure that b1 is never b
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue

                        co += bipart_coeff \
                            * gw_invariant(n, b_, [A_0 - 1, 1, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k, [n - i] + I2)   

                for b1 in range(1, b+1): # zero when b1 = 0
                    b2 = b - b1 

                    for k1 in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        ool += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1, [A_0 - 1, 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 2, I2)   

                for b1 in range(1, b+1): # exclude b1 = 0, which is zero, but would still cause infinite recursion
                    b2 = b - b1

                    for k in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        oor += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1 + 1, [A_0 - 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, [1] + I2) 

            result =  -co + ool - oor
            if self.debug:
                print(f"recursion c : {result}")
            return result   

        
        # recursion (d)
        if k >= 2:
            sum1 = 0
            sum2 = 0
            sum3 = 0
            
            for t in A.bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A, I1)    

                for i in range(0, n+1):
                    for b1 in range(0, (b+2)+1): # make sure that b1 is never b
                        b_ = 0
                        if ((b + 2) - b1) % 4 == 0:
                            b_ = ((b + 2) - b1) / 4
                            if b_ == 0:
                                continue
                        else:
                            continue
                        sum1 += bipart_coeff \
                            * gw_invariant(n, b_, [2, 2, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k-1, [n - i] + I2)
                
                for b1 in range(2, b+1): # zero when b1 = 0
                    b2 = (b + 2) - b1 

                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        sum2 += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+1, [2] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2+1, [2] + I2)
                    
                for b1 in range(2, b+1):
                    b2 = (b + 2) - b1
                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        
                        if b1 == b and k1 == k - 2:
                            continue
                            
                        sum3 += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+2, I1) \
                            * self.chiang_ogw_invariant(n, b2, k2, [2, 2] + I2)
            
            result = (sum1 - sum2 + sum3)/(self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([2, 2], n)))       
            if self.debug:
                print(f"recursion d: {result}")
            return result   

        print(f"unexpected situation is , b: {b}, k: {k}, A:{A.c_list}, l: {l}")
        return 0

In [56]:
# get value that satisfies the degree axiom 
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + 2*b + k + 2*l2 - k*n - 2*l2 * A2 # degree axiom
    if (m % (2*(A1 - 1))) == 0:
        return int(m / (2*(A1 - 1)))
    else:
        -1


In [57]:

chiang_ogw = ChiangOGWInvariant(debug=False)
w = chiang_ogw.chiang_ogw_invariant(3, 2, 0, ClassList.compress([2, 2], 3))
print(f"w:{w}")
#b == 2 and k == 0 and A == [2, 2]

w:-175/512


In [58]:
chiang_ogw = ChiangOGWInvariant(debug=False)
# k = 0, no boundary points
for b in range(0, 10):
    for l2 in [1, 3, 5]:
        A1 = 2
        A2 = 3
        l1 = get_l1(3, b, 0, l2, A1, A2)
        if l1 < 0 :
            continue
        A = [A1]*l1 + [A2]*l2
        chiang = chiang_ogw.chiang_ogw_invariant(3, b, 0, ClassList.compress(A, 3))
        print(f"b: {b}, A: {A}, chiang ogw: {chiang}")

b: 2, A: [3], chiang ogw: -5879/8192
b: 3, A: [2, 3], chiang ogw: 17637/65536
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
b: 4, A: [2, 2, 3], chiang ogw: -264555/1048576
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
b: 5, A: [2, 2, 2, 3], chiang ogw: 80294955/268435456
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

...

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[

In [59]:
chiang_ogw_a = ChiangOGWInvariant()
chiang_ogw_c = ChiangOGWInvariant(use_recursion_a=False)
# experiment k > 0

for b in range(1, 5):
    for k in range(0, 7):
        for l2 in range(0, 6):
            A1 = 2
            A2 = 3
        
            l1 = get_l1(3, b, k, l2, A1, A2)
            # l1 = 1
            if l1 < 0 :
                continue
            A = [A1]*l1 + [A2]*l2
            chiang_a = chiang_ogw_a.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            chiang_c = chiang_ogw_c.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))

            print(f"b:{b}, k:{k}, A:{A}, chiang_a: {chiang_a}, chiang_c: {chiang_c}")

b:1, k:0, A:[2], chiang_a: -93/1024, chiang_c: -93/1024
b:1, k:1, A:[], chiang_a: 3, chiang_c: 3
b:2, k:0, A:[2, 2], chiang_a: -175/512, chiang_c: -175/512
b:2, k:0, A:[3], chiang_a: -5879/8192, chiang_c: -5879/8192
b:2, k:1, A:[2], chiang_a: -9/16, chiang_c: -9/16
b:2, k:2, A:[], chiang_a: 2, chiang_c: 2
b:3, k:0, A:[2, 2, 2], chiang_a: 42837/131072, chiang_c: 42837/131072
b:3, k:0, A:[2, 3], chiang_a: 17637/65536, chiang_c: 17637/65536
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 2, 0]
b:3, k:1, A:[2, 2], chiang_a: 525/4096, chiang_c: 525/4096
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
b:3, k:1, A:[3], chiang_a: 0, chiang_c: 0
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
b:3, k:2, A:[2], chiang_a: 0, chiang_c: 0
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 1, 0]
used c, A:[0, 0, 2, 0]
b:3, k:3, A:[], chiang_a: -3/4, chiang_c: -3/4
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]
used c, A:[0, 0, 0, 1]


In [60]:
x, y = var('x, y')
solve([-1/4*x + 27/256 == 525/4096
      ],
      x)

#-1/4*x + 27/256, chiang_c: 525/4096



[x == (-93/1024)]

In [61]:
x, y = var('x, y')
solve([-256/175*x + 108/175 == 3/4
      ],
      x)



[x == (-93/1024)]

In [62]:
x, y = var('x, y')
solve([ -86272/30625*x + 61452/6125 == 41472/30625*x + 1275447/122500
      ],
      x)
#1/4096*(64*x - 27)*x + 351/2048*x - 61425/131072, chiang_c: 10707/65536*x - 61425/131072

[x == (-93/1024)]

In [63]:
x, y = var('x, y')
solve([  1/4096*(64*x - 27)*x + 351/2048*x - 61425/131072 == 10707/65536*x - 61425/131072
      ],
      x)


[x == (-93/1024), x == 0]

In [64]:
x, y = var('x, y')
solve([ 135/1024*x - 7875/32768 == 135/1024*x - 7875/32768
      ],
      x)

#chiang_a: 135/1024*x - 7875/32768, chiang_c: 135/1024*x - 7875/32768

[x == r1]

In [65]:
b:4, k:1, A:[2, 3],
            chiang_a: -3/16*x + 175/512,
                chiang_c: 81/1024*x - 4725/32768

SyntaxError: invalid syntax (<ipython-input-65-5e7567b4f044>, line 1)